#### Arain UnitTest-Finetuning датасет

Здесь рассматривается датасет [Arain UnitTest](https://huggingface.co/datasets/Arain/UnitTest-Finetuning). Код, представленный в данном датасете, написан на двух языках программирования: Python и Java. Данные представленны в виде prompt2code для файнтюнинга языковых моделей и обучения их пониманию написанного кода на разных downstream tasks. 

Запросы для генерации тестов разделяются на 2 вида: 

* Method-Test Dataset: The dataset can be used to train a model for test completion and test generation tasks in Agile Development (AD)

* Docstring-Test-Method Dataset: The dataset can be used to train a model for test completion, test generation tasks and functional code generation in Test-Driven Development (TDD)

В идеале бы получить именно первый вариант (не скачивал полные данные, пока что есть только примеры TDD), так как именно в нем запросы ЧЁТКО разделены на код входа и дополнительную информацию, в чём я не уверен касательно датасета типа TDD. (Если верить README датасета)

Посмотрим на формат данных.

In [ ]:
import json

def parse_json_input(path_json = './data/data_Arain_unitTest-FineTuning_example/tdd_python_few_shot_tdd1.json'):
	'''Функция считываем файл формата JSON по пути и преобразует его в dict'''
	with open(path_json, 'r') as file:
		json_dict = json.load(file)

	return json_dict

json_example = parse_json_input()


Запрос:

In [8]:
json_example[0]['conversations'][0]['value']

"You are a professional python software engineer proficient in utilizing the Test-Driven Development (TDD) methodology. Your development process consists of two steps: first, generating test cases based on natural language requirements, and second, creating functional code.\nCurrently, you're embarking on the first step and a unit test class for a requirement is under development, your task is to generate a new test method for this test class to test new aspects that have not been covered before.\nYou'll be provided with the following information:\n1. A development requirement described in natural language.\n2. Source code of unit test method that is already developed(With imports and dependencies).\nYou will ONLY return unit test code including necessary imports and dependencies, make sure it compile without errors, use reflection to invoke private methods, and won't test scenarios beyond the stated development requirement. \nNote that no additional explanations required.\n\nHere are 

Требуемый ответ:

In [9]:
json_example[0]['conversations'][1]['value']

"```python\nfrom pyamg.relaxation.relaxation import gauss_seidel, jacobi, \\\n    block_jacobi, block_gauss_seidel, jacobi_ne, schwarz, sor, \\\n    gauss_seidel_indexed, polynomial, gauss_seidel_ne, \\\n    gauss_seidel_nr, \\\n    jacobi_indexed, cf_jacobi, fc_jacobi, cf_block_jacobi, fc_block_jacobi\nfrom numpy.testing import TestCase, assert_almost_equal\nfrom scipy.sparse import spdiags, csr_matrix, bsr_matrix, eye\nimport numpy as np\n\nclass TestRelaxation(TestCase):\n    def test_gauss_seidel_indexed(self):\n        N = 1\n        A = spdiags([2*np.ones(N), -np.ones(N), -np.ones(N)], [0, -1, 1], N, N,\n                    format='csr')\n        x = np.arange(N).astype(np.float64)\n        b = np.zeros(N)\n        gauss_seidel_indexed(A, x, b, [0])\n        assert_almost_equal(x, np.array([0]))\n        N = 3\n        A = spdiags([2*np.ones(N), -np.ones(N), -np.ones(N)], [0, -1, 1], N, N,\n                    format='csr')\n        x = np.arange(N).astype(np.float64)\n        b 

Посмотрим на вид кода по запросу (англ.):

Вы являетесь профессиональным инженером-программистом на Python, владеющим методологией разработки через тестирование (TDD). Ваш процесс разработки состоит из двух этапов: во-первых, создание тестовых случаев на основе требований на естественном языке, а во-вторых, написание функционального кода.  
Сейчас вы находитесь на первом этапе и разрабатываете класс модульных тестов для одного из требований. Ваша задача — написать новый метод тестирования для проверки аспектов, которые еще не были покрыты.  

Вам предоставляется следующая информация:  
1. Требование для разработки, описанное на естественном языке.  
2. Исходный код метода модульного тестирования, уже реализованного (включая импорты и зависимости).  

Вы должны вернуть ТОЛЬКО код модульного теста, включая необходимые импорты и зависимости. Убедитесь, что код компилируется без ошибок, используйте рефлексию для вызова приватных методов и не тестируйте сценарии, выходящие за рамки указанного требования.  
Дополнительные пояснения не требуются.  

**Информация**:  
1. **Требование для разработки, описанное на естественном языке:**  
Реализовать метод для выполнения итерации Гаусса-Зейделя для решения линейной системы \(Ax = b\).  

   **Параметры:**  
   - `A`: матрица формата `csr_matrix` или `bsr_matrix`, разреженная матрица размером \(N \times N\).  
   - `x`: `ndarray`, начальное приближение решения (длина \(N\)).  
   - `b`: `ndarray`, правая часть уравнения (длина \(N\)).  
   - `iterations`: `int`, количество итераций.  
   - `sweep`: `{'forward', 'backward', 'symmetric'}`, направление обхода.  

   **Возвращаемое значение:**  
   - Ничего, так как \(x\) изменяется на месте.  

   **Пример:**  
   ```python
   from pyamg.relaxation.relaxation import gauss_seidel
   from pyamg.gallery import poisson
   import numpy as np

   A = poisson((10, 10), format='csr')
   x0 = np.zeros((A.shape[0], 1))
   b = np.ones((A.shape[0], 1))
   gauss_seidel(A, x0, b, iterations=10)

... и еще три примерных теста вида:

```python
class TestRelaxation(TestCase):
def test_gauss_seidel_bsr(self):
	sweeps = ['forward', 'backward', 'symmetric']
	cases = []
	for N in [1, 2, 3, 4]:
		cases.append(spdiags([2 * np.ones(N), -np.ones(N), -np.ones(N)],
								[0, -1, 1], N, N).tocsr())
	for A in cases:
		for sweep in sweeps:
			x = np.zeros(A.shape[0])
			b = np.arange(A.shape[0], dtype=float)
			gauss_seidel(A, x, b, sweep=sweep)
			assert_almost_equal(x, np.linalg.solve(A.toarray(), b))





Получим код теста:

In [ ]:
def code_format(code_string = json_example[0]['conversations'][1]['value']):
	'''Функция, которая сделает из строчки код в удобном читаемом формате и сохранит его в generated_code.py'''
	with open("generated_code.py", "w") as file:
		file.write(code_string)

code_format()


```python
from pyamg.relaxation.relaxation import gauss_seidel, jacobi, \
    block_jacobi, block_gauss_seidel, jacobi_ne, schwarz, sor, \
    gauss_seidel_indexed, polynomial, gauss_seidel_ne, \
    gauss_seidel_nr, \
    jacobi_indexed, cf_jacobi, fc_jacobi, cf_block_jacobi, fc_block_jacobi
from numpy.testing import TestCase, assert_almost_equal
from scipy.sparse import spdiags, csr_matrix, bsr_matrix, eye
import numpy as np

class TestRelaxation(TestCase):
    def test_gauss_seidel_indexed(self):
        N = 1
        A = spdiags([2*np.ones(N), -np.ones(N), -np.ones(N)], [0, -1, 1], N, N,
                    format='csr')
        x = np.arange(N).astype(np.float64)
        b = np.zeros(N)
        gauss_seidel_indexed(A, x, b, [0])
        assert_almost_equal(x, np.array([0]))
        N = 3
        A = spdiags([2*np.ones(N), -np.ones(N), -np.ones(N)], [0, -1, 1], N, N,
                    format='csr')
        x = np.arange(N).astype(np.float64)
        b = np.zeros(N)
        gauss_seidel_indexed(A, x, b, [0, 1, 2])
        assert_almost_equal(x, np.array([1.0/2.0, 5.0/4.0, 5.0/8.0]))
        N = 3
        A = spdiags([2*np.ones(N), -np.ones(N), -np.ones(N)], [0, -1, 1], N, N,
                    format='csr')
        x = np.arange(N).astype(np.float64)
        b = np.zeros(N)
        gauss_seidel_indexed(A, x, b, [2, 1, 0], sweep='backward')
        assert_almost_equal(x, np.array([1.0/2.0, 5.0/4.0, 5.0/8.0]))
        N = 3
        A = spdiags([2*np.ones(N), -np.ones(N), -np.ones(N)], [0, -1, 1], N, N,
                    format='csr')
        x = np.arange(N).astype(np.float64)
        b = np.zeros(N)
        gauss_seidel_indexed(A, x, b, [0, 1, 2], sweep='backward')
        assert_almost_equal(x, np.array([1.0/8.0, 1.0/4.0, 1.0/2.0]))
        N = 4
        A = spdiags([2*np.ones(N), -np.ones(N), -np.ones(N)], [0, -1, 1], N, N,
                    format='csr')
        x = np.ones(N)
        b = np.zeros(N)
        gauss_seidel_indexed(A, x, b, [0, 3])
        assert_almost_equal(x, np.array([1.0/2.0, 1.0, 1.0, 1.0/2.0]))
        N = 4
        A = spdiags([2*np.ones(N), -np.ones(N), -np.ones(N)], [0, -1, 1], N, N,
                    format='csr')
        x = np.ones(N)
        b = np.zeros(N)
        gauss_seidel_indexed(A, x, b, [0, 0])
        assert_almost_equal(x, np.array([1.0/2.0, 1.0, 1.0, 1.0]))
```


In [13]:
code_format(json_example[0]['conversations'][0]['value'])

#### Резюме

Данный датасет содерджит основные ключевые особенности:

* *Работает с промптами, не с кодом*. Это может противоречить поставленной задаче, так как изначально мы хотели бы генерировать тесты из кода, но никак не из запроса к языковой модели.

* Содержит *большое количество размечененных тестов* для двух языков программирования (Python + Java). 

#### ***Идея***

В принципе мысль идея как-то получить из запроса код входа и заранее релевантных тестов, чтобы как-то получить в итоге свзяку код <-> тесты. Стоит пристально рассмотреть, так как датасет действительно выглядит как релевантный для нашей задачи.

Помимо того в [описании](https://huggingface.co/datasets/Arain/UnitTest-Finetuning/blob/main/README.md) к датасету приложены виды промптов в зависимости от типа датасета (TDD/AD). Этот факт упрощает задачу, поскольку можно фактически одним махом получить код, несколько тестов и еще дополнительные комментарии из раздела описания задачи. (То есть не выбрасывать комментарии, а оставлять их для обработки естественного языка).

